In [7]:
import pandas as pd

c2n = pd.read_excel('CSA-to-NSA-2010.xlsx')
hinc = pd.read_excel('Vital Signs Indicator Median Household Income.xlsx')
z2c = pd.read_excel('Zip-to-CSA-2010.xls')
df = pd.read_csv('MDCOVID19_MASTER_ZIP_CODE_CASES.csv')
census = pd.read_csv('Census_Demographics_2010-2012.csv')

In [8]:
df = df.fillna(0)
df = df.drop('OBJECTID', axis=1)

df = df.transpose()

df.columns = df.iloc[0]
df = df.drop('ZIP_CODE')
df = df.reset_index()
df = df.rename(columns={'index':'Date'})

df['Date'] = df['Date'].map(lambda x: x.lstrip('F').lstrip('t').lstrip('o').lstrip('t')\
                            .lstrip('a').lstrip('l'))

dates = pd.DataFrame()
dates = df['Date'].str.split(pat='_', expand=True)
dates[0] = dates[0].map(lambda x: x.lstrip('0'))
dates['fdate'] = dates[2].str.cat(dates[0], sep='-').str.cat(dates[1],sep='-')

dates = dates['fdate']

df = df.drop('Date',axis=1)

df.insert(0,'Date',dates)

df['Date'] = pd.to_datetime(df['Date'])

print(df.head())
df.to_csv('COVID_Cleaned.csv')

ZIP_CODE       Date  20010.0  20015.0  20037.0  20074.0  20161.0  20185.0  \
0        2020-04-11      0.0      0.0      0.0      0.0      0.0      0.0   
1        2020-04-12      0.0      0.0      0.0      0.0      0.0      0.0   
2        2020-04-13      0.0      0.0      0.0      0.0      0.0      0.0   
3        2020-04-14      0.0      0.0      0.0      0.0      0.0      0.0   
4        2020-04-15      0.0      0.0      0.0      0.0      0.0      0.0   

ZIP_CODE  20437.0  20601.0  20602.0  ...  21911.0  21912.0  21913.0  21914.0  \
0             0.0     36.0     44.0  ...     14.0      0.0      0.0      0.0   
1             0.0     38.0     48.0  ...     21.0      0.0      0.0      0.0   
2             0.0     41.0     51.0  ...     21.0      0.0      0.0      0.0   
3             0.0     43.0     57.0  ...     25.0      0.0      0.0      0.0   
4             0.0     45.0     59.0  ...     30.0      0.0      0.0      0.0   

ZIP_CODE  21915.0  21917.0  21918.0  21919.0  21921.0  2

In [3]:
#pipeline rows:dates, columns:zips -> rows:zips, columns:zips-----------------
df = df.transpose()
df = df.reset_index()

df.columns = df.iloc[0]

df = df.rename(columns={'Date':'Zip Code'})
df1 = df.drop([0,0])

print(df1.head())
df1.to_csv('COVID_Cleaned_Transposed.csv')

0 Zip Code 2020-04-11 00:00:00 2020-04-12 00:00:00 2020-04-13 00:00:00  \
1    20010                   0                   0                   0   
2    20015                   0                   0                   0   
3    20037                   0                   0                   0   
4    20074                   0                   0                   0   
5    20161                   0                   0                   0   

0 2020-04-14 00:00:00 2020-04-15 00:00:00 2020-04-16 00:00:00  \
1                   0                   0                   0   
2                   0                   0                   0   
3                   0                   0                   0   
4                   0                   0                   0   
5                   0                   0                   0   

0 2020-04-17 00:00:00 2020-04-18 00:00:00 2020-04-19 00:00:00  ...  \
1                   0                   0                   0  ...   
2                   0   

In [4]:
#pipeline zips, CSAs, NSAs, MHINC, CSA pop, total COVID cases -> one file-----
zipsLatest = pd.DataFrame()

zipsLatest['Zip'] = df1['Zip Code'].astype(int)
zipsLatest['TotalCOVIDCases'] = df1.iloc[:,-1]

census = census[['CSA2010','tpop10']]
census = census.rename(columns={'CSA2010':'Community','tpop10':'Pop2010'})
#rename columns appropriately
z2c = z2c.rename(columns={'Zip2010':'Zip','CSA2010':'CSA'})
c2n = c2n.rename(columns={'CSA2010':'Community'})

#merge, rename Household income, CSA population, and CSAs-to-Neighborhoods
df = pd.merge_ordered(hinc, c2n, fill_method='ffill')
df = pd.merge_ordered(census,df,fill_method='ffill')
df = df.rename(columns={'2018 Data':'MHINC','NSA2010':'Neigh','Community':'CSA'})
df = df[['Neigh','CSA','Pop2010','MHINC']]

df = df.drop([0])
df.to_csv('test.csv')
#merge with Zip Codes
df = pd.merge_ordered(z2c,df,fill_method='ffill')

#drop commas from household income numbers, convert to float
df['MHINC'] = df['MHINC'].replace(',','',regex=True)
df['MHINC'] = df['MHINC'].astype(float)

#convert zips to ints
df['Zip'] = df['Zip'].astype(int)

#merge with latest covid cases
df = pd.merge_ordered(zipsLatest,df,fill_method='ffill')
df = df[['Neigh','CSA','Pop2010','Zip','MHINC','TotalCOVIDCases']]

#export as CSV
df.to_csv('MASTER_MERGED.csv',index=False)
print(df.head())

  Neigh  CSA  Pop2010    Zip  MHINC TotalCOVIDCases
0   NaN  NaN      NaN  20010    NaN               0
1   NaN  NaN      NaN  20015    NaN               0
2   NaN  NaN      NaN  20037    NaN               0
3   NaN  NaN      NaN  20074    NaN               0
4   NaN  NaN      NaN  20161    NaN               0


In [5]:
#pipeline COVID_Cleaned_Transposed -> Daily Increase--------------------------
cc = pd.read_csv('COVID_Cleaned.csv')

cc = cc.transpose()
cc = cc.drop('Unnamed: 0')
cc = cc.reset_index()

cc.columns = cc.iloc[0]

cc = cc.rename(columns={'Date':'Zip Code'})
cc = cc.drop([0,0]) 

cc1 = cc.loc[:,'2020-04-11':]\
        .diff(axis=1)\
        .drop(columns='2020-04-11')\
  
dates = cc['Zip Code']
cc1.insert(0,'Zip Code',dates)    

cc.to_csv('MD_COVID_NewCasesByDate.csv')
print(cc.head())

0 Zip Code 2020-04-11 2020-04-12 2020-04-13 2020-04-14 2020-04-15 2020-04-16  \
1  20010.0          0          0          0          0          0          0   
2  20015.0          0          0          0          0          0          0   
3  20037.0          0          0          0          0          0          0   
4  20074.0          0          0          0          0          0          0   
5  20161.0          0          0          0          0          0          0   

0 2020-04-17 2020-04-18 2020-04-19  ... 2021-04-16 2021-04-17 2021-04-18  \
1          0          0          0  ...          0          0          0   
2          0          0          0  ...          0          0          0   
3          0          0          0  ...          0          0          0   
4          0          0          0  ...          0          0          0   
5          0          0          0  ...          0          0          0   

0 2021-04-19 2021-04-20 2021-04-21 2021-04-22 2021-04-23 2021-

In [6]:
#pipeline COVID_Cleaned_Transposed -> Daily Increase--------------------------
cc = pd.read_csv('COVID_Cleaned.csv')

cc = cc.transpose()
cc = cc.drop('Unnamed: 0')
cc = cc.reset_index()

cc.columns = cc.iloc[0]

cc = cc.rename(columns={'Date':'Zip Code'})
cc = cc.drop([0,0]) 

cc1 = cc.loc[:,'2020-04-11':]\
        .diff(axis=1)\
        .drop(columns='2020-04-11')\
  
dates = cc['Zip Code']
cc1.insert(0,'Zip Code',dates)    

cc.to_csv('MD_COVID_NewCasesByDate.csv')
print(cc.head())

#pipeline -> Mean, median new cases, total cases
latestCases = pd.DataFrame()

latestCases['Zip Code'] = df1['Zip Code']
latestCases['Total Cases'] = df1.iloc[:,-1]

latestCases['Zip Code'] = latestCases['Zip Code'].astype(int)
latestCases['Total Cases'] = latestCases['Total Cases'].astype(int)

meanList = []
medianList = []

#loop through rows to calculate mean/median new cases per day per zip code
for x in range(0,len(cc.index)):
    #add values to list
    meanList.append(cc.iloc[x,2:].mean())
    medianList.append(cc.iloc[x,2:].median())

#add to latest cases
latestCases.insert(2,'meanNewCasesPerDay',meanList)
latestCases.insert(3,'medianNewCasesPerDay',medianList)

print(latestCases.head())
latestCases.to_csv('TotalMeanMedianCOVIDbyZip.csv')

0 Zip Code 2020-04-11 2020-04-12 2020-04-13 2020-04-14 2020-04-15 2020-04-16  \
1  20010.0          0          0          0          0          0          0   
2  20015.0          0          0          0          0          0          0   
3  20037.0          0          0          0          0          0          0   
4  20074.0          0          0          0          0          0          0   
5  20161.0          0          0          0          0          0          0   

0 2020-04-17 2020-04-18 2020-04-19  ... 2021-04-16 2021-04-17 2021-04-18  \
1          0          0          0  ...          0          0          0   
2          0          0          0  ...          0          0          0   
3          0          0          0  ...          0          0          0   
4          0          0          0  ...          0          0          0   
5          0          0          0  ...          0          0          0   

0 2021-04-19 2021-04-20 2021-04-21 2021-04-22 2021-04-23 2021-